### Instalar librerias y dependencias necesarias para el proyecto

In [1]:
!pip install python-dotenv --upgrade
!pip install boto3 --upgrade
!pip install pyaml --upgrade
!pip install Jinja2 --upgrade
!pip install opensearch-py
!pip install tqdm


  Attempting uninstall: python-dotenv
    Found existing installation: python-dotenv 1.0.0
    Uninstalling python-dotenv-1.0.0:
      Successfully uninstalled python-dotenv-1.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 83.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.131
    Uninstalling botocore-1.34.131:
      Successfully uninstalled botocore-1.34.131
  Attempting uninstall: s3transfer
    Found existing installation: s3transfer 0.6.2
    Uninstalling s3transfer-0.6.2:
      Successfully uninstalled s3transfer-0.6.2
  Attempting uninstall: boto3
    Found existing installation: boto3 1.26.100
    Uninstalling boto3-1.26.100:
      Successfully uninstalled boto3-1.26.100
ERROR: pip's dependency resolver does not currently take into account

### Importar librerias y dependencias

In [31]:
import os
import sys
import logging
import boto3
import botocore
import yaml
import json
import requests
import zipfile

from tqdm import tqdm
from typing import Callable
from time import time, sleep

from jinja2 import Template, Environment
from dotenv import load_dotenv, find_dotenv
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from opensearchpy.helpers import bulk

# Configurar el logging
logging.basicConfig(level=logging.DEBUG, 
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    handlers=[
                        logging.FileHandler("drugs.log"),
                        logging.StreamHandler()
                    ]
                   )


### Cargar variables de entorno y archivos de configuración

In [32]:
# Carga variables entorno y archivos de configuración
if not(load_dotenv(find_dotenv())):
    sys.exit('No se puede continuar porque no se han definido las variables de entorno para el  proyecto!')
        
# Cargar archivo de configuraciones .yaml
try:
    # Definir una función para cargar variables de entorno
    def get_env_variable(name):
        return os.getenv(name)

    # Crear un entorno Jinja2 con la función de carga de variables de entorno
    env = Environment()

    # Agregar la función de entorno al entorno Jinja2
    env.globals['env'] = get_env_variable

    with open('config.yaml', 'r') as file:
        template_content = file.read()
    
     # Cargar el contexto YAML
    yaml_data = yaml.safe_load(template_content)
    
     # Renderizar la plantilla con el contexto
    template = env.from_string(template_content)
    rendered_content = template.render(**yaml_data)
    
    # Cargar el YAML renderizado
    yaml_data = yaml.safe_load(rendered_content)
    
    print("Archivo de configuracion cargado exitosamente.")
except FileNotFoundError:
    print("Error: El archivo de configuración .yaml no se encuentra disponible")
except yaml.YAMLError as exc:
    print(f"Error al analizar el archivo YAML: {exc}")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

Archivo de configuracion cargado exitosamente.


In [33]:
#Valida que se hayan definido las variables de configuración y las variables de entorno
try:
    # Cargar variables de entorno
    aws_region = os.environ['aws_region']
    aws_access_key_id = os.environ['aws_access_key_id']
    aws_secret_access_key = os.environ['aws_secret_access_key']
    
    # Cargar variables de configuración
    project = yaml_data.get('Project', {})
    project_name = project.get('name')
    url_dataset = project.get('url_dataset')
    
    open_search = yaml_data.get('OpenSearch', {}) 
    service = open_search.get('service')
    port = open_search.get('port')
    use_ssl = open_search.get('use_ssl')
    verify_certs = open_search.get('verify_certs')
    pool_maxsize = open_search.get('pool_maxsize')
    timeout = open_search.get('timeout')
    collection_name = open_search.get('Collection',{}).get('name')
    collection_index = open_search.get('Collection',{}).get('index')
   
    if(aws_region is None):
        raise Exception('No se ha definido la variable de entorno: aws_region')
    elif(aws_access_key_id is None):
        raise Exception('No se ha definido la variable de entorno: aws_access_key_id')
    elif(aws_secret_access_key is None):
        raise Exception('No se ha definido la variable de entorno: aws_secret_access_key')
    if(project_name is None):
        raise Exception('No se ha definido la variable de configuración del proyecto : project_name')
    elif(url_dataset is None):
        raise Exception('No se ha definido la variable de configuración del proyecto : url_dataset')
    elif(service is None):
        raise Exception('No se ha definido la variable de configuración de openSearch-AWS : service')
    elif(port is None):
        raise Exception('No se ha definido la variable de configuración de openSearch-AWS : port')
    elif(use_ssl is None):
        raise Exception('No se ha definido la variable de configuración de openSearch-AWS : use_ssl')
    elif(verify_certs is None):
        raise Exception('No se ha definido la variable de configuración de openSearch-AWS : verify_certs')
    elif(pool_maxsize is None):
        raise Exception('No se ha definido la variable de configuración de openSearch-AWS : pool_maxsize')
    elif(collection_name is None):
        raise Exception('No se ha definido la variable de configuración de openSearch-AWS : collection_name')
    elif(collection_index is None):
        raise Exception('No se ha definido la variable de configuración de openSearch-AWS : collection_index')
    
    # Define variables de entorno y variables globales
    DATASET_FOLDER = project_name + '/dataset'
    
     # Crea directorio para almacenar los datasets
    if not(os.path.exists(DATASET_FOLDER)):
        !mkdir -p {DATASET_FOLDER}
        print('Directorio creado exitosamente!!')
    
except Exception as ex:
    print(ex)
    print(f'Error en la linea No. {ex.__traceback__.tb_lineno}')

### Definición de Funciones

In [63]:
#  Decorador para medir el tiempo de ejecución de la función
def measure_time(func:Callable)->Callable:
    '''
    args: one argument, is the function that we want to decorated(e,g functions that:
    build the model, train the model, optimize the model, ...etc)
    '''
    # define the wrapper function that executes the original function(e,g arg func)
    def wrapper(*args, **kwargs)->float:
        '''
        the wrapper function have the same argument as the object(e,g arg)function
        '''
        start_time=time()
        # call the object function to process and manipulate ...
        result=func(*args, **kwargs)
        end_time=time()

        processing_time=end_time-start_time
        print(f'tiempo de procesamiento {processing_time:.2f} segundos')
        return result

    return wrapper

# Función para validar si una URL es válida
def is_url_valid(url : str) -> bool:
    try:
        response = requests.head(url)
        return response.status_code == 200
    except Exception:
        return False

# Función para descargar archivos desde repositorio openFDA
@measure_time
def download_files(url_file : str, chunk_size: int = 1024):
    try:
        # Hacer la solicitud para descargar el archivo .zip
        response = requests.get(url_file, stream=True)
        if(response.status_code != 200):
            raise Exception(f"Error en la solicitud HTTP: {url_file}| respuesta: {response.status_code}")
        
        zip_filename = os.path.join(DATASET_FOLDER, url_file.split('/')[-1])
        
        total_size = int(response.headers.get('content-length', 0))
        
        # Guardar el archivo .zip descargado
        with open(zip_filename, 'wb') as file:
            file.write(response.content)
            #for data in tqdm(response.iter_content(chunk_size=chunk_size), total=total_size//chunk_size, unit='KB'):
            #    file.write(data)
                    
        logging.debug("Archivo guardado como: {}".format(zip_filename))
        
        # Extraer el contenido del archivo .zip
        with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
            zip_ref.extractall(DATASET_FOLDER)
        logging.debug("Archivo extraído: {}".format(zip_filename))

        # Eliminar el archivo .zip descargado
        os.remove(zip_filename)
        logging.debug("Archivo .zip eliminado {}".format(zip_filename))
        
    except zipfile.BadZipFile:
        logging.error("El archivo no es un archivo zip válido: {}".format(zip_filename))
    except FileNotFoundError:
        logging.error("El archivo no fue encontrado: {}".format(zip_filename))
    except Exception as ex:
        logging.error("Ocurrió un error al extraer el archivo {}: {}".format(zip_filename, e))
        
def get_data_drugs(json_file):
    try:
        actions = []
        if(json_file):
            #
            data = json.load(json_file)
            last_updated = data['meta']['last_updated']
            
            for item in data['results']:
                item['last_updated'] = last_updated
                
                # Crear un nuevo diccionario con la estructura deseada y fusionar openfda si existe
                results = {**{k: v for k, v in item.items() if k != "openfda"}, **item.get("openfda", {})}
                action = {
                    "_index": collection_index,
                    "_id": results['id'],  # Usar el atributo 'id' como ID del documento
                    "_source": results
                }
                actions.append(action)
        return actions
            
    except Exception as ex:
        logging.error("Ocurrió un error al leer los datos {} ".format(ex))
        print(f"Ocurrió un error al leer los datos {ex}")
        print(ex.__traceback__.tb_lineno)


### Descargar dataset desde OpenFDA

In [35]:
# Descargar dataset desde OpenFDA
try:
    response = requests.get(url_dataset)
    if(response.status_code != 200):
        raise Exception(f"Error en la solicitud HTTP. Código de respuesta: {response.status_code}")
        
    data = response.json()
    
    # Obtener las particiones
    partitions = data.get('results', {}).get('drug', {}).get('label', {}).get('partitions', [])

    boo = False
    if isinstance(partitions, list):
        total_files = len(partitions)
        with tqdm(total=total_files, desc="Procesando descarga") as pbar:
            for partition in partitions:
                file_url = partition.get('file', '')
                if file_url and is_url_valid(file_url):
                    print(f"Descargando archivo desde: {file_url}")
                    download_files(file_url)
                    pbar.update(1)
    else:
        print("El nodo 'partitions' no es una lista.")
        
except Exception as ex:
    print(ex)
    



Procesando descarga:   0%|          | 0/12 [00:00<?, ?it/s]

Descargando archivo desde: https://download.open.fda.gov/drug/label/drug-label-0001-of-0012.json.zip


Procesando descarga:   8%|▊         | 1/12 [00:06<01:10,  6.45s/it]

tiempo de procesamiento 6.39 segundos
Descargando archivo desde: https://download.open.fda.gov/drug/label/drug-label-0002-of-0012.json.zip


Procesando descarga:  17%|█▋        | 2/12 [00:12<01:01,  6.15s/it]

...

tiempo de procesamiento 8.50 segundos
Descargando archivo desde: https://download.open.fda.gov/drug/label/drug-label-0011-of-0012.json.zip


Procesando descarga:  92%|█████████▏| 11/12 [01:17<00:07,  7.52s/it]

tiempo de procesamiento 6.12 segundos
Descargando archivo desde: https://download.open.fda.gov/drug/label/drug-label-0012-of-0012.json.zip


Procesando descarga: 100%|██████████| 12/12 [01:20<00:00,  6.72s/it]

tiempo de procesamiento 2.88 segundos


### Cargar información al servicio de OpenSearh Serverless AWS

In [65]:
try:
    # Establecer conexión con OpenSearch-AWS-Serverless
    session = boto3.Session(aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=aws_region)
    client = session.client('opensearchserverless')
    credentials = session.get_credentials()
    awsauth = AWSV4SignerAuth(credentials, aws_region, service)  
    response = client.batch_get_collection(names=[collection_name])
    
    #sleep(2)

    # Extraer el endpoint de la colección desde el response
    aws_host = (response['collectionDetails'][0]['collectionEndpoint'])
    aws_host = aws_host.replace("https://", "")

    # Construir el cliente de OpenSearch
    client = OpenSearch(hosts=[{'host': aws_host, 'port': port}],
                        http_auth=awsauth,
                        use_ssl=use_ssl,
                        verify_certs=verify_certs,
                        connection_class=RequestsHttpConnection,
                        timeout=timeout
                        )
    
    # Leer archivos JSON
    json_files = [pos_json for pos_json in os.listdir(DATASET_FOLDER) if pos_json.endswith('.json')]

    # Leer el contenido de cada archivo JSON
    data_set = []
    for json_file in json_files:
        with open(os.path.join(DATASET_FOLDER, json_file), 'r') as f:
            data_set = get_data_drugs(f)
            #print(json.dumps(data_set, indent=2))
        
        # Eviar una colección de datos de forma masiva usando bulk de OpenSearch.helper
        success, failed = bulk(client, data_set)
        print('success : ', success)
        print('failed : ', failed)
        
        break;

    # Insertar un documento en OpenSearch
    '''
    try:
        for doc in data:
            response = client.index(index=collection_index, body=doc)

    except Exception as ex:
        print(ex.error)
    '''    
    
except Exception as ex:
    print(f"Ocurrió un error al procesar los archivos json : {ex}")
    print(ex.__traceback__.tb_lineno)
    

success :  3
success :  []


In [30]:
! mv example.json ./drugs-generative/dataset

Overwriting config.yaml
